In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/ML_KR_Mekhtieva/data/'

Mounted at /content/drive


In [ ]:
# Загрузка данных
train_df = pd.read_parquet(FILE_PATH+'train_data.pqt')
test_df = pd.read_parquet(FILE_PATH+'test_data.pqt')
print(train_df.shape, test_df.shape)

# Подсчет числа заполненных значений в каждой строке
train_df['filled_cols'] = train_df.drop(['id', 'date', 'start_cluster', 'end_cluster'], axis=1).count(axis=1)
test_df['filled_cols'] = test_df.drop(['id', 'date', 'start_cluster'], axis=1).count(axis=1)

# Наиболее вероятный конечный кластер для заданного начального кластера
clusters = {
 '{other}': '{other}',
 '{}': '{}',
 '{α, β}': '{α, β}',
 '{α, γ}': '{α, γ}',
 '{α, δ}': '{α}',
 '{α, ε, η}': '{α, ε, η}',
 '{α, ε, θ}': '{α, ε, θ}',
 '{α, ε, ψ}': '{α, ε, ψ}',
 '{α, ε}': '{α, ε}',
 '{α, η}': '{α, η}',
 '{α, θ}': '{α, θ}',
 '{α, λ}': '{α, λ}',
 '{α, μ}': '{α, μ}',
 '{α, π}': '{other}',
 '{α, ψ}': '{α, ψ}',
 '{α}': '{α}',
 '{λ}': '{α, λ}'
}

# Заполнение начального кластера в month_6 значениями из month_5
test_df['start_cluster'].fillna(method='ffill', inplace=True)

# Предсказание конечного кластера по матрице вероятностей
train_df['end_cluster_1'] = train_df['start_cluster'].map(clusters)
test_df['end_cluster_1'] = test_df['start_cluster'].map(clusters)

# Определение категориальных признаков
cat_cols = [
    "channel_code",
    "city",
    "city_type",
    "okved",
    "segment",
    "start_cluster",
    "index_city_code",
    "ogrn_month",
    "ogrn_year",
    'end_cluster_1'
]

# Преобразование категориальных признаков в числовые значения с помощью LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

# Создание выборок для валидации и обучения
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

# Преобразование целевой переменной с помощью LabelEncoder
y = le.fit_transform(y)

# Список фичей с наибольшей значимостью
top_features = ['filled_cols', 'end_cluster_1', 'start_cluster', 'okved', 'index_city_code', 'channel_code', 'city',
                'balance_amt_min', 'segment', 'balance_amt_max', 'ogrn_days_end_quarter', 'sum_of_paym_1y',
                'ogrn_days_end_month', 'cnt_a_oper_1m', 'min_founderpres', 'ogrn_exist_months', 'ogrn_month',
                'ft_registration_date', 'ogrn_year', 'sum_of_paym_6m', 'max_founderpres', 'sum_deb_e_oper_3m',
                'balance_amt_avg', 'sum_of_paym_2m', 'sum_cred_e_oper_3m', 'cnt_days_deb_e_oper_3m', 'balance_amt_day_avg',
                'cnt_cred_e_oper_3m', 'sum_cred_e_oper_1m', 'sum_cred_h_oper_3m', 'cnt_days_cred_e_oper_3m', 'cnt_deb_e_oper_3m',
                'sum_deb_h_oper_3m', 'cnt_a_oper_3m', 'sum_deb_f_oper_3m', 'cnt_cred_e_oper_1m', 'sum_deb_g_oper_3m', 'sum_deb_e_oper_1m',
                'cnt_cred_h_oper_3m', 'cnt_deb_h_oper_3m', 'sum_deb_d_oper_3m', 'cnt_deb_e_oper_1m', 'sum_deb_f_oper_1m', 'sum_deb_h_oper_1m',
                'cnt_days_cred_e_oper_1m', 'cnt_deb_g_oper_3m', 'cnt_days_cred_h_oper_3m', 'sum_cred_g_oper_3m', 'sum_c_oper_3m', 'sum_a_oper_3m',
                'sum_cred_h_oper_1m', 'city_type', 'cnt_deb_d_oper_3m', 'sum_deb_d_oper_1m', 'cnt_days_deb_e_oper_1m', 'cnt_deb_f_oper_3m',
                'sum_deb_g_oper_1m', 'cnt_days_deb_h_oper_3m', 'cnt_days_deb_f_oper_3m', 'cnt_c_oper_3m', 'cnt_days_deb_g_oper_3m',
                'cnt_b_oper_1m', 'cnt_days_deb_h_oper_1m', 'cnt_days_cred_h_oper_1m', 'cnt_deb_f_oper_1m', 'sum_cred_f_oper_3m',
                'cnt_days_cred_g_oper_1m', 'cnt_cred_g_oper_3m', 'sum_cred_d_oper_3m', 'sum_b_oper_3m', 'sum_c_oper_1m', 'sum_a_oper_1m',
                'cnt_deb_g_oper_1m', 'cnt_deb_h_oper_1m', 'cnt_b_oper_3m', 'cnt_days_cred_f_oper_1m', 'cnt_c_oper_1m', 'cnt_days_deb_g_oper_1m',
                'cnt_days_deb_f_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_h_oper_1m', 'cnt_cred_d_oper_1m', 'sum_b_oper_1m', 'cnt_deb_d_oper_1m',
                'cnt_cred_f_oper_3m', 'cnt_cred_d_oper_3m', 'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'sum_cred_d_oper_1m', 'cnt_cred_g_oper_1m',
                'cnt_days_cred_g_oper_3m', 'cnt_days_cred_f_oper_3m']

selected_features = top_features[:53]

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='median')
X = imputer.fit_transform(X)
test_df_imputed = imputer.fit_transform(test_df[selected_features])

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

(600000, 93) (290120, 92)


In [ ]:
# Обучение модели Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [ ]:
# Загрузка весов кластеров
cluster_weights = pd.read_excel(FILE_PATH + 'cluster_weights.xlsx').set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

# Добавление отсутствующих классов в weights_dict
all_classes = rf_model.classes_
for cls in all_classes:
    if cls not in weights_dict:
        weights_dict[cls] = 1  # или любое другое значение по умолчанию

# Функция для взвешенной метрики ROC AUC
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    # Получаем веса для всех меток
    unnorm_weights = np.array([weights_dict.get(label, 1) for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()

    # Считаем метрику для всех меток
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels, multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

# Проверка работы модели на валидационной выборке
y_pred_proba = rf_model.predict_proba(x_val)
print("Метрика на валидации (Random Forest Classifier):", weighted_roc_auc(y_val, y_pred_proba, rf_model.classes_, weights_dict))

Метрика на валидации (Random Forest Classifier): 0.90017966797762


In [ ]:
# Обучение модели на всей выборке
rf_model_full = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1)
rf_model_full.fit(X, y)

KeyboardInterrupt: 

In [ ]:
# Создание DataFrame с предсказанными вероятностями для тестовой выборки
test_pred_proba = rf_model_full.predict_proba(test_df_imputed)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=rf_model_full.classes_)

# Сортировка по классам
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

# Проверка формы предсказанного DataFrame
print("Форма предсказанного DataFrame:", test_pred_proba_df.shape)
print(test_pred_proba_df.head(10))

# Загрузка шаблона для отправки
sample_submission_df = pd.read_csv(FILE_PATH + "sample_submission.csv")

# Добавление предсказаний в sample_submission
sample_submission_df[sorted_classes] = test_pred_proba_df

# Запись предсказаний в файл
sample_submission_df.to_csv("rf_result.csv", index=False)